# OpenAPI

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

In [1]:
%pip install jsonref

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Import the libraries

In [2]:
import os
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import OpenApiTool, OpenApiConnectionAuthDetails, OpenApiConnectionSecurityScheme, MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [3]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup the OpenAPI Tool

In [4]:
# Enter TripAdvisor connection names here
tripadvisor = "tripadvisor"  # Replace with your TripAdvisor connection name

# Load the OpenAPI specification from a JSON file
with open('../data/tripadvisor/tripadvisor_openapi.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

# Extract the connection list.
conn_list = project_client.connections.list()
tripadvisor_conn_id = ""

# Find the connection IDs for TripAdvisor
for conn in conn_list:
    if conn.name == tripadvisor:
        tripadvisor_conn_id = conn.id
  
# Print the connection names
print(f"TripAdvisor Connection ID: {tripadvisor_conn_id}")

# Create the OpenAPI tool
auth = OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=tripadvisor_conn_id))
tripadvisor_tool = OpenApiTool(
                    name=tripadvisor, 
                    spec=openapi_spec, 
                    description="get hotel and restaurant reviews of a location", 
                    auth=auth
                )

TripAdvisor Connection ID: /subscriptions/73047bc3-f020-4930-b16d-fe5c9dd3c73e/resourceGroups/AzureStandardAgent/providers/Microsoft.CognitiveServices/accounts/foundykx3l/projects/projectkx3l/connections/tripadvisor


## Create the agent

In [5]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-tripadvisor-agent",  # Name of the agent
    instructions="You are a helpful agent that can search TripAdvisor for hotel and restaurant reviews",  # Instructions for the agent
    tools=tripadvisor_tool.definitions,  # Tools available to the agent
    tool_resources=tripadvisor_tool.resources,  # Resources for the tools
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_t0WwkDxbu2HmaseRapbNZPXV


## Create thread

In [6]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_Ui4DPRaqUz3hzPz3QNQICESf


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [7]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

In [8]:
def run_agent_details(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Fetch run steps to get the details of the agent run
    run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                function_details = call.get("function", {})
                if function_details:
                    print(f"    Function name: {function_details.get('name')}")
        print()  # add an extra newline between steps

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [9]:
user_input = """
  What restaurants can you recommend in New York from TripAdvisor? 
"""
run_agent_details(user_input)

Created message, ID: msg_alxVOXAD1R7Htjt3ItRRh09n
Run finished with status: completed
Step step_RGjOLWEbSbDQa6Uj1FhmoSfu status: completed

Step step_X6y2zCbf3SJqxJflEmrAy8UG status: completed
  Tool calls:
    Tool Call ID: call_40pPBemo5p5eyYTt3DvSQFLZ
    Type: openapi
    Function name: tripadvisor_searchForLocations

Agent Response: Here are some restaurant recommendations in New York City from TripAdvisor:

1. **New York Luncheonette**
   - Address: 135 E 50th St, Suite 1, Randolph House, New York City, NY 10022-7504, United States

2. **Zuma New York**
   - Address: 261 Madison Avenue, Between 38th and 39th street, New York City, NY 10016, United States

3. **Nusr-Et Steakhouse New York**
   - Address: 60 W 53rd St, Ground Level, New York City, NY 10019-6106, United States

Feel free to let me know if you need more information or further recommendations!


In [10]:
user_input = """
  What do people say about Katz’s Delicatessen in New York? 
  Give me 5 reviews and provide citations for each review.
"""
run_agent(user_input)

Created message, ID: msg_k3OdnCEhDkTo09g3h2n80YDV
Run finished with status: completed
Agent Response: Here are 5 reviews for Katz’s Delicatessen in New York:

1. **Review by: pinceme_iam_dreaming from Bentonville, Arkansas**
   - Rating: 3/5
   - Title: "The sandwich was huge came and with pickles. It was $28.95+ tax (affordable) It just wasn't that tasty. it was fun 🙂"
   - Review: "We had to try world famous Katz' pastrami sandwich. It was busy but not packed, we found a table no problem. Door staff were unfriendly but not mean or rude. The sandwich-making guy was great and gave us a sample of other meats. Overall a good experience at a famous place; however, we've had much better pastrami at a lot of other places. We've been there and no real reason to go back."
   - [Link to review](https://www.tripadvisor.com/ShowUserReviews-g60763-d425787-r1016658275-Reviews-Katz_s_Deli-New_York_City_New_York.html?m=66827#review1016658275)

2. **Review by: U2016SZmarcoss from San Juan, Puerto Ric

In [11]:
user_input = """ 
  Provide me a summarized description about the Ritz Carlton, New York.
"""
run_agent_details(user_input)

Created message, ID: msg_xCzUmZ6OfZLnPqjjQyL11IkT
Run finished with status: completed
Step step_fxUKawnCWuzm1WNlBOjvCICi status: completed

Step step_TkhXYQ3U8okyKdnS2WZCNYHA status: completed
  Tool calls:
    Tool Call ID: call_8WFLTrRTyxYMK8O2qZpEPY9r
    Type: openapi
    Function name: tripadvisor_getLocationDetails

Step step_LJr5cCPc9oxMr8yl3zipdK7i status: completed
  Tool calls:
    Tool Call ID: call_t1Y0Uth7YPi5OaKKYg1NAM41
    Type: openapi
    Function name: tripadvisor_searchForLocations

Agent Response: ### The Ritz-Carlton New York, Central Park

**Description:**
Boldly transformed and redesigned, The Ritz-Carlton New York, Central Park reimagines luxury in Midtown Manhattan, drawing inspiration from the sights, sounds, and colors of Central Park and the chic residential style of a city penthouse. Located steps from the city's renowned cultural attractions, this 253-room hotel is iconic and award-winning. The all-day gastro lounge Contour, The Ritz-Carlton Club® Lounge,

## Cleanup resources

In [12]:
# Cleanup resources

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted agent
